# Подход 1: градиентный бустинг "в лоб"
Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

In [7]:
# 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 
#    Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

import pandas

train = pandas.read_csv('./data/features.csv', index_col='match_id')
test = pandas.read_csv('./data/features_test.csv', index_col='match_id')

train.drop(['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']
           , axis=1, inplace=True)

# 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает 
#    число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, 
#    и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

maxCount = 0
C = train.count()
for x in range(0, len(C)):
    if C[x] > maxCount:
        maxCount = C[x]
print('maxCount =',maxCount) # // 97230

# признаки, имеющие пропуски
for idx,item in enumerate(C):
    if item < maxCount:
        print( C.keys()[idx],': ', maxCount-item)


maxCount = 97230
first_blood_time :  19553
first_blood_team :  19553
first_blood_player1 :  19553
first_blood_player2 :  43987
radiant_bottle_time :  15691
radiant_courier_time :  692
radiant_flying_courier_time :  27479
radiant_first_ward_time :  1836
dire_bottle_time :  16143
dire_courier_time :  676
dire_flying_courier_time :  26098
dire_first_ward_time :  1826


In [8]:
# 4. Какой столбец содержит целевую переменную? Запишите его название.

y_train = train['radiant_win']

del train['radiant_win']

# 3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным 
#    для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. 
#    Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — 
#    в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. 
#    Также есть и другие подходы — например, замена пропуска на среднее значение признака. 

X_train = train.fillna(0)
X_test = test.fillna(0)


In [9]:
# 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями 
#    на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), 
#    не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, 
#    и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество 
#    градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное 
#    количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). 
#    Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, 
#    или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
import datetime
import numpy as np

# Разделение выборки для кроссвалидации
cv = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []
# Кол-ва деревьев для модели
nums = [10, 20, 30, 40, 50, 60]

for n in nums:
    print('Trees: ', str(n))
    model = GradientBoostingClassifier(n_estimators=n, random_state=42)
    start_time = datetime.datetime.now()
    model_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    print('Time spent:', datetime.datetime.now() - start_time)
    print(model_scores)
    scores.append(np.mean(model_scores))

Trees:  10
Time spent: 0:00:27.978600
[0.66383799 0.66635457 0.66360048 0.66529818 0.66516222]
Trees:  20
Time spent: 0:00:54.957144
[0.68083889 0.68272733 0.67969876 0.6834932  0.6855512 ]
Trees:  30
Time spent: 0:01:18.056464
[0.68892093 0.68934663 0.68712298 0.69180598 0.69283583]
Trees:  40
Time spent: 0:01:41.158786
[0.69264125 0.69335305 0.69153074 0.69586466 0.69680392]
Trees:  50
Time spent: 0:02:05.564182
[0.69627399 0.69747879 0.69470891 0.69921915 0.69979097]
Trees:  60
Time spent: 0:02:30.645617
[0.69904111 0.69974831 0.69741008 0.70187297 0.70252411]


# Отчет

**1.Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?**

• first_blood_player1/first_blood_player2 (игроки причастные к событию first_blood) первой крови нет в первые 5 минут игры

• first_blood_time (игровое время первой крови) - первой крови нет в первые 5 минут игры

• first_blood_team (команда, совершившая первую кровь) - аналогично

• dire_bottle_time/radiant_bottle_time (время первого приобретения командой предмета "bottle") - команда не приобретает предмет "bottle" в первые 5 минут игры

• dire_first_ward_time/radiant_first_ward_time (время установки командой первого "наблюдателя") - команда не ставит "наблюдателя" в первые 5 минут игры

• dire_courier_time/radiant_courier_time (время приобретения предмета "courier") - команда не приобретает предмет "courier" в первые 5 минут игры

• dire_flying_courier_time/radiant_flying_courier_time (время приобретения предмета "flying_courier") - команда не покупает "flying_courier" в первые 5 минут игры


**2.Как называется столбец, содержащий целевую переменную?**

radiant_win - победа команды radiant и проигрыш комады dire

**3.Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.**

Кросс-валидация для градиентного бустинга с 30 деревьями заняла 0:01:15.397313. Показатель AUC-ROC равен 0.69.

**4.Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?**

Увеличение количества деревьев увеличивает показатель метрики AUC-ROC. Для достижения большего качества имеет смысл использовать более 30 деревьев. Для ускорения обучения можно уменьшить глубину деревьев (параметр max_depth).


# Подход 2: Логистическая регрессия
Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.

In [22]:
# 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью 
#    кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший 
#    параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством 
#    градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по 
#    сравнению с градиентным бустингом?
import pandas
features = pandas.read_csv('./data/features.csv', index_col='match_id')
X = features.iloc[:,:-6]
X = X.fillna(0)
y = features['radiant_win']
X.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [13]:
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.metrics
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

XX = sklearn.preprocessing.StandardScaler().fit_transform(X)

fold = sklearn.cross_validation.KFold(n=len(XX), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,20)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX, y)
max_qual=0
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
    if score.mean_validation_score>max_qual:
        max_qual=score.mean_validation_score
print("Best param: ", gs.best_params_,"Best quality: ", max_qual)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Wall time: 2min 49s
Quality 0.7162096360086553 for {'C': 0.001}
Quality 0.7163550390674079 for {'C': 0.05357894736842105}
Quality 0.7163506207526636 for {'C': 0.1061578947368421}
Quality 0.7163498860052757 for {'C': 0.15873684210526315}
Quality 0.7163493841091112 for {'C': 0.2113157894736842}
Quality 0.716348913884599 for {'C': 0.26389473684210524}
Quality 0.716348549503929 for {'C': 0.3164736842105263}
Quality 0.7163482826608616 for {'C': 0.36905263157894735}
Quality 0.7163478759004407 for {'C': 0.4216315789473684}
Quality 0.7163479713103026 for {'C': 0.47421052631578947}
Quality 0.7163479141800596 for {'C': 0.5267894736842105}
Quality 0.7163477447100794 for {'C': 0.5793684210526315}
Quality 0.7163476408932814 for {'C': 0.6319473684210526}
Quality 0.7163476154766693 for {'C': 0.6845263157894736}
Quality 0.7163474863198411 for {'C': 0.7371052631578947}
Quality 0.7163474100495975 for {'C': 0.7896842105263158}
Quality 0.716347456704766 for {'C': 0.8422631578947368}
Quality 0.716347352958

Наилучшее качество **Quality = 0.716355** достигается при параметре **C = 0.0536**. 

Oно превосходит качество полученное при градиентном бустинге. Градиентый бустинг хорошо показывает себя на задачах где признаки хорошо сгруппированны в пространстве. Логистическая регрессия же лучше работает когда объекты слабо сгруппированы в пространстве.

In [14]:
# 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является 
#    хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, 
#    d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической 
#    регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете 
#    это объяснить?

def drop_from(X):
    drops = ["{}{}_hero".format(c, i) for c in 'rd' for i in range(1,6)]
    X2 = X.drop('lobby_type', axis=1).drop(drops, axis=1)
    return X2

X2 = drop_from(X)

In [15]:
XX2 = sklearn.preprocessing.StandardScaler().fit_transform(X2)

fold = sklearn.cross_validation.KFold(n=len(XX2), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,20)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX2, y)
max_qual=0
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
    if score.mean_validation_score>max_qual:
        max_qual=score.mean_validation_score
print("Best param: ", gs.best_params_,"Best quality: ", max_qual)

Wall time: 2min 29s
Quality 0.7162333103715237 for {'C': 0.001}
Quality 0.7163870909247205 for {'C': 0.05357894736842105}
Quality 0.7163834689703076 for {'C': 0.1061578947368421}
Quality 0.7163825010628666 for {'C': 0.15873684210526315}
Quality 0.7163817068944599 for {'C': 0.2113157894736842}
Quality 0.7163817494545419 for {'C': 0.26389473684210524}
Quality 0.7163814868685365 for {'C': 0.3164736842105263}
Quality 0.7163813640418073 for {'C': 0.36905263157894735}
Quality 0.7163812116132652 for {'C': 0.4216315789473684}
Quality 0.7163807900891609 for {'C': 0.47421052631578947}
Quality 0.7163806248671812 for {'C': 0.5267894736842105}
Quality 0.7163805189512752 for {'C': 0.5793684210526315}
Quality 0.7163804977403702 for {'C': 0.6319473684210526}
Quality 0.716380398227788 for {'C': 0.6845263157894736}
Quality 0.7163803770691148 for {'C': 0.7371052631578947}
Quality 0.7163803579815369 for {'C': 0.7896842105263158}
Quality 0.71638041307506 for {'C': 0.8422631578947368}
Quality 0.716380368608

Удаление категориальных признаков не повлияло на качество предсказания и оптимальное значение параметра C.  Из этого можно сделать вывод, что в предыдущей модели эти признаки никак не влияли на результат предсказания, модель смогла распознать этот шум.

In [16]:
# 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно 
#    герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из 
#    них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует 
#    в данной игре (вам может пригодиться фукнция unique или value_counts).

drops = ["{}{}_hero".format(c, i) for c in 'rd' for i in range(1,6)]
heroes = np.unique(features[drops].fillna(0))
unique_heroes = len(heroes)
unique_heroes

108

Существует 108 различных идентификтаоров героев.

In [18]:
# 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных
#    героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, 
#    если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете 
#    найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы 
#    использовали во втором пункте данного этапа.

def pick_from(X):
    X_pick = np.zeros((X.shape[0], unique_heroes))
    for i, match_id in enumerate(X.index):
        for p in range(1,6):
            X_pick[i, np.where(heroes == X.loc[match_id, 'r%d_hero' % p])] = 1
            X_pick[i, np.where(heroes == X.loc[match_id, 'd%d_hero' % p])] = -1
    return X_pick

X_pick = pick_from(X)
X4 = X2.join(pandas.DataFrame(X_pick, index=features.index))

In [19]:
# 5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра 
#    регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

XX4 = sklearn.preprocessing.StandardScaler().fit_transform(X4)

fold = sklearn.cross_validation.KFold(n=len(XX4), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,20)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX4, y)
max_qual=0
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
    if score.mean_validation_score>max_qual:
        max_qual=score.mean_validation_score
print("Best param: ", gs.best_params_,"Best quality: ", max_qual)

Wall time: 4min 30s
Quality 0.7516014700448687 for {'C': 0.001}
Quality 0.7518686023830814 for {'C': 0.05357894736842105}
Quality 0.7518615140622452 for {'C': 0.1061578947368421}
Quality 0.7518593510973987 for {'C': 0.15873684210526315}
Quality 0.7518591606784638 for {'C': 0.2113157894736842}
Quality 0.7518585949211656 for {'C': 0.26389473684210524}
Quality 0.7518578725437954 for {'C': 0.3164736842105263}
Quality 0.7518574087301215 for {'C': 0.36905263157894735}
Quality 0.7518572350827064 for {'C': 0.4216315789473684}
Quality 0.7518570931330799 for {'C': 0.47421052631578947}
Quality 0.7518569830605741 for {'C': 0.5267894736842105}
Quality 0.7518568284556554 for {'C': 0.5793684210526315}
Quality 0.7518567627753631 for {'C': 0.6319473684210526}
Quality 0.75185657419508 for {'C': 0.6845263157894736}
Quality 0.7518564449250293 for {'C': 0.7371052631578947}
Quality 0.7518564173782895 for {'C': 0.7896842105263158}
Quality 0.7518562436907335 for {'C': 0.8422631578947368}
Quality 0.75185617166

Качество предсказаний классификатора выросло **c 0.716355 до 0.751869**. До текущего момента перечисленные в задании категориальные признаки не вносили значительного вклада при построении класификатора ввиду особенностей класификатора. Теперь же эти данные адаптированы для использования классификатором.

In [20]:
# 6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных 
#    моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — 
#    находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

features_test = pandas.read_csv('./features_test.csv', index_col='match_id')
X_test = features_test
X_test = X_test.fillna(0)
X2_test = drop_from(X_test)
X4_test = X2_test.join(pandas.DataFrame(pick_from(X_test), index=features_test.index))
X4_test.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,98,99,100,101,102,103,104,105,106,107
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,4,1103,1089,8,0,1,9,3,1183,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1430293357,2,556,570,1,0,0,9,4,1194,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,1430301774,2,751,808,1,0,0,13,2,421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13,1430323933,3,708,903,1,1,1,11,2,672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
16,1430331112,4,1259,661,4,0,0,9,5,1703,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
XX4 = sklearn.preprocessing.StandardScaler().fit_transform(X4)
logreg = sklearn.linear_model.LogisticRegression(C=gs.best_params_['C'])
logreg.fit(XX4, y)

XX4_test = sklearn.preprocessing.StandardScaler().fit_transform(X4_test)
res = logreg.predict_proba(XX4_test)
print(res.min(), res.max(),res.mean())
print(res)

0.003542405240436697 0.9964575947595633 0.5
[[0.17544807 0.82455193]
 [0.24195397 0.75804603]
 [0.81270878 0.18729122]
 ...
 [0.76834837 0.23165163]
 [0.36821547 0.63178453]
 [0.57342602 0.42657398]]


Предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой и их среднее арифметическое равно 0.5. Минимальное значение - 0.0035, максимальное – 0.9965

# Отчет:
**1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом? **

Наилучшее качество 0.716 достигается при C=0.0536, оно превосходит качество полученное при градиентном бустинге. Градиентый бустинг хорошо показывает себя на задачах где признаки хорошо сгруппированны в пространстве, т.к. ориентирован на нахождение таких многомерных скоплений объектов. Логистическая регрессия же позволяет найти гиперплоскость максимально разделяющую объекты двух классов - она лучше работает когда объекты слабо сгруппированы в пространстве. Логистическая регрессия по скорости не превзогла градиентный бустинг.

**2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение? **

Качество классификатора практически не изменилось, это объяснимо тем что линейная модель плохо работает с категориальными данными, без их специальной обработки. Потому исключение таких данных из выборки - не повлияло на качество.

**3. Сколько различных идентификаторов героев существует в данной игре? **

Существует 108 различных идентификтаоров героев.

**4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить? **

Качество предсказаний классификатора выросло до 0.752. До текущего момента перечисленные в задании категориальные признаки не вносили значительного вклада при построении класификатора ввиду особенностей класификатора. Теперь же эти данные адаптированы для использования классификатором.

**5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов? **

Минимальное значение - 0.0035, максимальное – 0.9965, среднее арифметическое - 0.5
